<a href="https://colab.research.google.com/github/csml1020gk/csml1030-capstone-project/blob/main/small_dataset_aug5_gouri_k_spacy_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, I will be working with a small text  dataset ( page 1 of a pdf ) to extract a term from the text and save it's span and label as a line in the training data file.

Once this model works, we can try to generalize it to work over text extracted from several pages within the same PDF.

We can then try to scale it up to work with

1. 1 entire pdf 
2. all pdfs   

At the end, we will develop a baseline model to predict the entity on  a test PDF 



In [ ]:
#installations
!pip install spacy


In [ ]:
#download language models 
!python -m spacy download en_core_web_sm
import spacy
nlp = spacy.load("en_core_web_sm")
#working with a larger model can be slower but can impact accuracy 

     |████████████████████████████████| 12.0 MB 27.8 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#imports
import os 
import pandas as pd
from pandas import DataFrame
import json
from spacy.matcher import Matcher, PhraseMatcher
from spacy.lang.en import English
import re

In [ ]:
#functions
def listToString(s): 
    # initialize an empty string
    str1 = "" 
    # traverse in the string  
    for ele in s: 
      str1 += ele
    # return string  
    return str1 



In [ ]:
#os
base_path = '/content/drive/MyDrive/week4'
save_path = '/content/drive/MyDrive/week4'
os.chdir(base_path)
os.getcwd()

'/content/drive/MyDrive/week4'

In [ ]:
os.listdir()
file_text = open(os.path.join(base_path, 'small_dataset.txt'),"r")
file_list = file_text.readlines()
file_list

['text\n',
 '0\n',
 "WELL CANADA LTD.Corporate lnformation Summaryas at November 25,2013Law Firm:Siskinds LLPTel5',t9 672-212'.1Fax519 672-9296Personnel:Graeme SperrynJennifer A. BoltonsolicitorclerkLAW FIRM INFORMATIONFile No.:840423Record No.:840423Accountinq No.:840423File Opening Date:November 6, 2013French Name:Date of lncorporation:November 4,2013Leqal Structure:Private CorporationJurisdiction:OntarioOntario Corporation No.:2394556Nature of Business:Contract Siqnins Authoritv:Unanimous Shareholder Aqreement:N/yAny Other Shareholder Aqreement:N/vTrade nameGENERAL INFORMATIONlsactive?Y/nKeep annua! resolutions current?Y/nLast an nual meeting/resolutionsAre signatures outstanding?N/vMinute Book- held by law firm?Y/nMinute Book- box number:Minute BookIY/nSTATUS AND RECORDS\n",
 '\n']

In [ ]:
#convert list to dataframe 

df_extracted_pages = DataFrame(file_list, columns =['text'])

In [ ]:
df_extracted_pages
#One page looks like this 
#row 0 - text 
#row1 - \n
#row2 - the page text as a string
#row 3 - \n 
#every page in our complete extracted text would follow this format


,text
0,text\n
1,0\n
2,WELL CANADA LTD.Corporate lnformation Summarya...
3,\n


In [ ]:
df_extracted_pages['text'][2]

"WELL CANADA LTD.Corporate lnformation Summaryas at November 25,2013Law Firm:Siskinds LLPTel5',t9 672-212'.1Fax519 672-9296Personnel:Graeme SperrynJennifer A. BoltonsolicitorclerkLAW FIRM INFORMATIONFile No.:840423Record No.:840423Accountinq No.:840423File Opening Date:November 6, 2013French Name:Date of lncorporation:November 4,2013Leqal Structure:Private CorporationJurisdiction:OntarioOntario Corporation No.:2394556Nature of Business:Contract Siqnins Authoritv:Unanimous Shareholder Aqreement:N/yAny Other Shareholder Aqreement:N/vTrade nameGENERAL INFORMATIONlsactive?Y/nKeep annua! resolutions current?Y/nLast an nual meeting/resolutionsAre signatures outstanding?N/vMinute Book- held by law firm?Y/nMinute Book- box number:Minute BookIY/nSTATUS AND RECORDS\n"

In [ ]:
#dataframe to txt
#get the filename minus extension
df_extracted_pages.to_csv(os.path.join(save_path, "small_data.txt"), sep ='\n')
#this is a text file and can be opened with google docs 
#the extension is .pdf, but not to confuse , it is a text file 
#pages are numbered 
#can be renamed 


In [ ]:
#create a processed Doc object for the document.
#input is the text file 
file_name = os.path.join(save_path, "small_data.txt")
file_text = open(file_name).read()
file_doc = nlp(file_text)
#the text file is converted into a processed doc file 

Statistical analysis - visualizations will be prepared on the full text. not here 



In [ ]:
#tokenization - identify basic units 
for token in file_doc:
  print (token, token.idx)
#preserves the starting index of the tokens


 0
text 1

 5
0 6

 7
" 8
text 9

 13
" 14

 15
1 16

 17
" 18
0 19

 20
" 21

 22
2 23

 24
" 25
WELL 26
CANADA 31
LTD.Corporate 38
lnformation 52
Summaryas 64
at 74
November 77
25,2013Law 86
Firm 97
: 101
Siskinds 102
LLPTel5',t9 111
672 123
- 126
212'.1Fax519 127
672 140
- 143
9296Personnel 144
: 157
Graeme 158
SperrynJennifer 165
A. 181
BoltonsolicitorclerkLAW 184
FIRM 208
INFORMATIONFile 213
No.:840423Record 229
No.:840423Accountinq 246
No.:840423File 267
Opening 282
Date 290
: 294
November 295
6 304
, 305
2013French 307
Name 318
: 322
Date 323
of 328
lncorporation 331
: 344
November 345
4,2013Leqal 354
Structure 366
: 375
Private 376
CorporationJurisdiction 384
: 407
OntarioOntario 408
Corporation 423
No.:2394556Nature 435
of 453
Business 456
: 464
Contract 465
Siqnins 474
Authoritv 482
: 491
Unanimous 492
Shareholder 502
Aqreement 514
: 523
N 524
/ 525
yAny 526
Other 531
Shareholder 537
Aqreement 549
: 558
N 559
/ 560
vTrade 561
nameGENERAL 568
INFORMATIONlsactive?Y 580
/ 601
n

In [ ]:
#attributes of each token
for token in file_doc:
  print (token, token.idx, token.text_with_ws,
    token.is_alpha, token.is_punct, token.is_space,
      token.shape_, token.is_stop)
#the tokenizer can be customized to our use case 


 0 
 False False True 
 False
text 1 text True False False xxxx False

 5 
 False False True 
 False
0 6 0 False False False d False

 7 
 False False True 
 False
" 8 " False True False " False
text 9 text True False False xxxx False

 13 
 False False True 
 False
" 14 " False True False " False

 15 
 False False True 
 False
1 16 1 False False False d False

 17 
 False False True 
 False
" 18 " False True False " False
0 19 0 False False False d False

 20 
 False False True 
 False
" 21 " False True False " False

 22 
 False False True 
 False
2 23 2 False False False d False

 24 
 False False True 
 False
" 25 " False True False " False
WELL 26 WELL  True False False XXXX True
CANADA 31 CANADA  True False False XXXX False
LTD.Corporate 38 LTD.Corporate  False False False XXX.Xxxxx False
lnformation 52 lnformation  True False False xxxx False
Summaryas 64 Summaryas  True False False Xxxxx False
at 74 at  True False False xx True
November 77 November  True False False Xxxxx Fal

In [ ]:
#statistical analysis 
from collections import Counter
words = [token.text for token in file_doc
  if not token.is_stop and not token.is_punct]
word_freq = Counter(words)
common_words = word_freq.most_common(5)
print (common_words)
unique_words = [word for (word, freq) in word_freq.items() if freq == 1]
print (unique_words)

[('\n', 14), ('November', 3), ('text', 2), ('0', 2), ('672', 2)]
['1', '2', 'CANADA', 'LTD.Corporate', 'lnformation', 'Summaryas', '25,2013Law', 'Firm', 'Siskinds', "LLPTel5',t9", "212'.1Fax519", '9296Personnel', 'Graeme', 'SperrynJennifer', 'A.', 'BoltonsolicitorclerkLAW', 'FIRM', 'INFORMATIONFile', 'No.:840423Record', 'No.:840423Accountinq', 'No.:840423File', 'Opening', '6', '2013French', 'lncorporation', '4,2013Leqal', 'Structure', 'Private', 'CorporationJurisdiction', 'OntarioOntario', 'Corporation', 'No.:2394556Nature', 'Business', 'Contract', 'Siqnins', 'Authoritv', 'Unanimous', 'yAny', 'vTrade', 'nameGENERAL', 'INFORMATIONlsactive?Y', 'nKeep', 'annua', 'resolutions', 'current?Y', 'nLast', 'nual', 'meeting', 'resolutionsAre', 'signatures', 'outstanding?N', 'vMinute', 'held', 'law', 'firm?Y', 'nMinute', 'box', 'number', 'Minute', 'BookIY', 'nSTATUS', 'RECORDS', '3']


In [ ]:
#create the training data to teach the named entity model
#TEXTS is a list of sentences in the page 
#find all mentions of WELL CANADA LTD. so we can create training data to teach a model to recognize them as COM 
#write a pattern which matches  WELL CANADA LTD. 






#file_name = os.path.join(save_path, "small_data.txt")
#TEXTS = open(file_name).read()
#matcher = PhraseMatcher(nlp.vocab, attr = "LOWER")

# Token which matches WELL CANADA LTD.
#pattern = [{"TEXT": "WELL CANADA LTD."}]

# Add patterns to the matcher and check the result
#matcher.add("COM", None, pattern)
#for doc in nlp.pipe(TEXTS):
#    print([doc[start:end] for match_id, start, end in matcher(doc)])





import spacy
from spacy.matcher import PhraseMatcher
corpus = df_extracted_pages['text'][2]#string 
nlp = spacy.load("en_core_web_sm")
doc = nlp(corpus)
matcher = PhraseMatcher(nlp.vocab)
terms = ["WELL CANADA"]#term to match 

train_data = open(os.path.join(save_path, '/content/drive/MyDrive/week4/ent.csv'), 'w')
# Only run nlp.make_doc to speed things up
patterns = [nlp.make_doc(text) for text in terms]
matcher.add("TerminologyList", patterns)#add the pattern to the matcher object 

matches = matcher(doc)
annotations = []#saved list of annotations, one in this case 
list_str = "("
for match_id, start, end in matches:
    span = doc[start:end]
    #list_str = span.text + "," + str(start) + "," +  str(end) + "\n"
    annotations.append((span.text, {'entities': [(start, end, 'COM')]}))
    annotations_str = str(annotations) + "\n"
    #write to file 
    train_data.write(str(annotations_str))
train_data.close()
#training data is in spacy format 

In [ ]:
print(annotations)
#WELL CANADA appears only once in this document , at the beginning 

[('WELL CANADA', {'entities': [(0, 2, 'COM')]})]


In [ ]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

In [ ]:
#create your custom entity data for the input text where the named entity is to be identified by the model during the testing period

TRAIN_DATA = annotations

In [ ]:
#define the variables required for the training model
model = None
output_dir=Path(save_path)
n_iter=10
#to save time,  only tried ten iterations 

In [ ]:
#load a blank model for the process to carry out the NER action and set up the pipeline with only NER using create_pipe function
#load the model

if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

#the pipleline can be expanded later to include other steps 
    

Created blank 'en' model


In [ ]:
for _, ann in TRAIN_DATA:
    for ent in ann.get('entities'):# we  only have one entity 
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)
    #in ten interations, the loss comes down to 68%
    


  0%|          | 0/1 [00:00<?, ?it/s]

{'ner': 1.0000000447034836}
{'ner': 0.9551979899406433}
{'ner': 0.9488962888717651}
{'ner': 0.8623597919940948}
{'ner': 0.8236702084541321}
{'ner': 0.7574411556124687}
{'ner': 0.6925287395715714}


100%|██████████| 1/1 [00:00<00:00, 40.50it/s]

{'ner': 0.6846142262220383}
{'ner': 0.6839388981461525}
{'ner': 0.6845945939421654}


In [ ]:
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(os.path.join(output_dir, 'nlp_model'))
    print("Saved model to", output_dir)

#save the model 

Saved model to /content/drive/MyDrive/week4


In [ ]:

try_model = spacy.load(os.path.join(output_dir, 'nlp_model'))#load the  trained model 

In [ ]:
try_model.entity.labels#our model can only predict COM labels at the moment 

('COM',)

In [ ]:
#input text file as string 
test_file = open(os.path.join(base_path, "test_data.txt"), "r")
file_str = listToString(test_file.readlines())

test_doc = try_model(file_str)

#print(test_doc)
for ent in test_doc.ents:
  print(f'{ent.label_:{20}} - {ent.text}')
#the ner model could not find any entities becuse of the large losses. we have very small training data too 

# The training data must have many examples in order to generate a good ner model that can predict entities from test data. 